In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

print(tf.__version__)


2.18.0


In [2]:
train_dir = r"E:\Folder kuliah\smstr7\TA1\klasifikasi-buah\dataset\train"
val_dir   = r"E:\Folder kuliah\smstr7\TA1\klasifikasi-buah\dataset\val"

img_size = (128, 128)
batch_size = 32


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

valid_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

class_count = len(train_gen.class_indices)
print("Detected Classes:", train_gen.class_indices)


Found 5763 images belonging to 10 classes.
Found 1240 images belonging to 10 classes.
Detected Classes: {'Banana__Healthy': 0, 'Banana__Rotten': 1, 'Carrot__Healthy': 2, 'Carrot__Rotten': 3, 'Cucumber__Healthy': 4, 'Cucumber__Rotten': 5, 'Potato__Healthy': 6, 'Potato__Rotten': 7, 'Tomato__Healthy': 8, 'Tomato__Rotten': 9}


In [4]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(128, 128, 3)
)

base_model.trainable = False  # Freeze CNN layer

model = models.Sequential([
    base_model,
    layers.GlobalMaxPooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(class_count, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 512)            │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
epochs = 10

history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=epochs
)

C:\PythonPackages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 12/181 ━━━━━━━━━━━━━━━━━━━━ 1:53 670ms/step - accuracy: 0.0875 - loss: 2.9452

c:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


 20/181 ━━━━━━━━━━━━━━━━━━━━ 1:44 650ms/step - accuracy: 0.1136 - loss: 2.8152

In [ ]:
# ✅ Save Model setelah selesai training
model.save("model-3.keras")
print("Model berhasil disimpan ✅")


Model berhasil disimpan ✅


In [ ]:
import json

label_map = train_gen.class_indices
with open("label_map.json", "w") as f:
    json.dump(label_map, f)

print("Label map berhasil disimpan ✅")


Label map berhasil disimpan ✅
